In [ ]:
!pip install langchain
!pip install langchain_community
!pip install chromadb
!pip install unstructured
!pip install pdfminer.six
!pip install pillow_heif
!pip install unstructured_inference
!apt-get install poppler-utils
!pip install unstructured[local-inference]
!pip install gradio

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

!ollama pull llama3.1
!ollama serve

In [ ]:
from langchain.llms import Ollama
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.vectorstores import Chroma
import numpy as np
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
import gradio as gr
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.ollama import OllamaEmbeddings
from langchain.chains import RetrievalQA

In [ ]:
# Initialize the language model
llm = Ollama(model="llama3.1")

In [ ]:
# # Example usage of the language model
# response = llm.invoke("Why is the sky blue?")
# print(response)

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("Engineering Order.pdf")
pages = loader.load()

In [ ]:
print("The document contains",len(pages),"pages\n")
print("----------------------------------------\n")
firstpage = pages[0]
print("Page 1 of the document is as follows:\n")
print(firstpage.page_content[0:250])

In [ ]:
# Initialize a text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=750
)
splits = text_splitter.split_documents(pages)
print("Number of Splits:", len(splits))

In [ ]:
embedding = OllamaEmbeddings(model="llama3.1")
from langchain.vectorstores import Chroma
persist_directory = 'chroma/'
!rm -rf ./docs/chroma

print("Vectorization Process Started...")
try:
    vectordb = Chroma.from_documents(
        documents=splits,
        embedding=embedding,
        persist_directory=persist_directory
    )
    print("Vectorization Process Completed!")
except Exception as e:
    print("Error during vectorization:", str(e))

vectordb._collection.count()

In [ ]:
# Initialize the language model
llm = Ollama(model="llama3.1")

In [ ]:
from langchain.chains import RetrievalQA

In [ ]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [ ]:
# Run chain
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [ ]:
question = "What is this document about?"
result = qa_chain({"query": question})
# result["result"]
print(result)

In [ ]:
# from langchain.memory import ConversationBufferMemory
# memory = ConversationBufferMemory(
#     memory_key="chat_history",
#     return_messages=True
# )

In [ ]:
# from langchain.chains import ConversationalRetrievalChain
# retriever=vectordb.as_retriever()
# qa = ConversationalRetrievalChain.from_llm(
#     llm,
#     retriever=retriever,
#     memory=memory
# )

In [ ]:
# def alternatingly_agree(question, history):
#         return result

# gr.ChatInterface(alternatingly_agree).launch()